___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [7]:
import requests
from pandas import DataFrame

requests.get("https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange").status_code

200

2. Отримати курс долара за останній рік.

In [2]:
from datetime import date, timedelta


url = "https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json"
year_rates = []
today = date.today()

for i in range(365):
    new_date = today - timedelta(days=i)
    r = requests.get(url, params={'valcode' : 'USD', 'date' : new_date.strftime('%Y%m%d')}).json()
    year_rates.append(r[0])

3. Вивести середнє значення та відхилення курсу за кожний місяць.

_Cортування курсів по місяцях_

In [3]:
months_rates = {}

for rate in year_rates:
    month = rate['exchangedate'].split('.')[1]
    months_rates.setdefault(month, []).append(rate) 

_Cереднє значення за кожний місяць_

In [16]:
avarage_months_rates = {}

for month, rates in months_rates.items():
    rates_sum = sum([rate['rate'] for rate in rates])
    avarage_months_rates[month] = rates_sum / len(rates)

DataFrame.from_dict(avarage_months_rates, orient='index', columns=['Середнє значення за кожний місяць'])

,Середнє значення за кожний місяць
08,27.157071
07,27.214548
06,27.239873
05,27.602848
04,27.930007
03,27.795558
02,27.884661
01,28.219771
12,28.169719
11,28.310153


_Відхилення курсу за кожний місяц_

In [18]:
months_deviations = {}

for month, month_rates in months_rates.items():
    result = 0
    for rate_data in month_rates:
        result += (rate_data['rate'] - avarage_months_rates[month]) ** 2

    months_deviations[month] = (result / len(month_rates)) ** 0.5

DataFrame.from_dict(months_deviations, orient='index', columns=['Відхилення курсу за кожний місяць:'])

,Відхилення курсу за кожний місяць:
08,0.290207
07,0.153294
06,0.153461
05,0.118493
04,0.089305
03,0.113636
02,0.104395
01,0.121151
12,0.245707
11,0.148351


4. Дану інформацію записати у файл за допомогою pickle.

In [19]:
import pickle

with open('rates.pickle', 'wb') as f:
    pickle.dump({'avarage month rates' : avarage_months_rates, 'months deviations' : months_deviations}, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

_Створення бази даних_

In [106]:
import sqlite3

shop_db = sqlite3.connect('shop.db')
curs = shop_db.cursor()
sql_req = """ 
    CREATE TABLE medications (
        sku INTEGER,
        name TEXT,
        price INTEGER
    )
 """
curs.execute(sql_req)
shop_db.commit()
shop_db.close()

_Заповнення бази даними_

In [115]:
from bs4 import BeautifulSoup
import requests


shop_db = sqlite3.connect('shop.db')
curs = shop_db.cursor()

# знаходимо к-сть сторінок
url = 'https://smallpacking.agrosem.ua/shop'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
pages = int(soup.find_all('a', class_='page-numbers')[-2].text)

# проходимо по всіх сторінках і знаходимо посилання на всі продукти сторінки
for page in range(1, pages + 1):
    r = requests.get(f'{url}/page/{page}/')
    soup = BeautifulSoup(r.text, 'lxml')
    products_elements = soup.find_all('div', class_="product-from-category-container")
    products_links = [el.a['href'] for el in products_elements]
    
    # проходимо по всіх посиланнях продуктів, заносимо дані в базу даних
    for link in products_links:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')        
        
        sku = int(soup.find('span', class_='sku').text.split()[1])
        name = soup.find('div', class_='atributes-description').h2.text
        price = soup.find('span', class_='regular-price').text
        price = int(price.replace('грн', '').replace(' ', ''))  

        sql_req = f""" 
            INSERT INTO medications(sku, name, price)
            VALUES ({sku}, "{name}", {price})
        """
        curs.execute(sql_req)

shop_db.commit()
shop_db.close()

_Відображення даних_

In [116]:
con = sqlite3.connect("shop.db")
curs = con.cursor()

data = curs.execute("SELECT * FROM medications").fetchall()
curs.execute("PRAGMA table_info(medications)")
columns = [row[1] for row in curs.fetchall()]

DataFrame(data, columns=columns)

,sku,name,price
0,28164,Ferticare Hydro 6-14-30 + micro,1900
1,66353,Ferticare Kombi 14-11-25 + micro,1700
2,39566,MagPlon K (нітрат калію),1200
3,69156,Pulan 34.4,7500
4,74266,Saletrosan®26,4400
...,...,...,...
114,5656841,Добриво для тепличних овочів,415
115,2217672,Добриво для троянд,155
116,8362626,Добриво для вічнозелених дерев та кущів,155
117,375738,Добриво для газону,999


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

_Підключення до бази даних_

In [124]:
import sqlite3

db = sqlite3.connect('tysql.sqlite')
curs = db.cursor()

1. Виведіть інформацію про дану базу.

In [125]:
curs.execute('select sqlite_version()')
curs.fetchall()

[('3.28.0',)]

2. Виведіть перелік всіх таблиць.

In [127]:
curs.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
DataFrame(curs.fetchall(), columns=['tables'])

,tables
0,Customers
1,OrderItems
2,Orders
3,Products
4,Vendors


3. Список всіх cust_id з таблиці Customers table.

In [128]:
curs.execute("SELECT cust_id FROM Customers")

DataFrame(curs.fetchall(), columns=['cust_id'])

,cust_id
0,1000000001
1,1000000002
2,1000000003
3,1000000004
4,1000000005


4. Всю таблицю Customers table

In [129]:
data = curs.execute("SELECT * FROM Customers").fetchall()
curs.execute("PRAGMA table_info(Customers)")
columns = [row[1] for row in curs.fetchall()]

DataFrame(data, columns=columns)

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


5. Список клієнтів (cust_names) відсортованих від Z до A

In [130]:
data = curs.execute("SELECT cust_name FROM Customers ORDER BY cust_name DESC").fetchall()
DataFrame(data, columns=['клієнти'])

,клієнти
0,Village Toys
1,The Toy Store
2,Kids Place
3,Fun4All
4,Fun4All


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [131]:
sql_req = """
            SELECT Customers.cust_id, Orders.order_num, Customers.cust_name, Orders.order_date
            FROM Customers
            INNER JOIN Orders
            ON Customers.cust_id = Orders.cust_id
            ORDER BY Customers.cust_name, Orders.order_date
"""
DataFrame(curs.execute(sql_req).fetchall(), columns=('cust_id', 'order_num', 'cust_name', 'order_date'))

,cust_id,order_num,cust_name,order_date
0,1000000003,20006,Fun4All,2012-01-12
1,1000000004,20007,Fun4All,2012-01-30
2,1000000005,20008,The Toy Store,2012-02-03
3,1000000001,20009,Village Toys,2012-02-08
4,1000000001,20005,Village Toys,2012-05-01


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [132]:
columns = 'order_item, quantity, item_price'
sql_req = f"""
            SELECT {columns}
            FROM OrderItems
            ORDER BY quantity DESC, item_price DESC
"""
DataFrame(curs.execute(sql_req).fetchall(), columns=columns.split(', '))

,order_item,quantity,item_price
0,1,250,2.49
1,2,250,2.49
2,3,250,2.49
3,2,100,10.99
4,1,100,5.49
5,2,100,2.99
6,3,100,2.99
7,4,100,2.99
8,1,50,11.49
9,5,50,4.49


8. Товар (з таблиці Products), ціна якого становить 9.49

In [133]:
data = curs.execute("SELECT * FROM Products WHERE prod_price = 9.49").fetchall()
curs.execute("PRAGMA table_info(Products)")
columns = [row[1] for row in curs.fetchall()]

DataFrame(data, columns=columns)

,prod_id,vend_id,prod_name,prod_price,prod_desc
0,RYL01,FNG01,King doll,9.49,12 inch king doll with royal garments and crown
1,RYL02,FNG01,Queen doll,9.49,12 inch queen doll with royal garments and crown


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [134]:
data = curs.execute("SELECT prod_name, prod_price FROM Products WHERE prod_price BETWEEN 3 AND 6").fetchall()
DataFrame(data, columns=('prod_name', 'prod_price'))

,prod_name,prod_price
0,8 inch teddy bear,5.99
1,Fish bean bag toy,3.49
2,Bird bean bag toy,3.49
3,Rabbit bean bag toy,3.49
4,Raggedy Ann,4.99


10. Кількість товару, що було продано

In [135]:
data = curs.execute('SELECT SUM(quantity) FROM OrderItems').fetchall()
DataFrame(data, columns=['Продано товару'])

,Продано товару
0,1430


11. Кількість найменувань товару, ціна якого більша за 4

In [136]:
sql_req = """
            SELECT COUNT(prod_price)
            FROM Products
            WHERE prod_price > 4
"""
DataFrame(curs.execute(sql_req).fetchall(), columns=['Кількість товарів'])

,Кількість товарів
0,6


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [137]:
data = curs.execute('SELECT * FROM Products ORDER BY prod_price DESC').fetchall()
curs.execute("PRAGMA table_info(Products)")
columns = [row[1] for row in curs.fetchall()]

DataFrame(data[:3], columns=columns)

,prod_id,vend_id,prod_name,prod_price,prod_desc
0,BR03,BRS01,18 inch teddy bear,11.99,"18 inch teddy bear, comes with cap and jacket"
1,RYL01,FNG01,King doll,9.49,12 inch king doll with royal garments and crown
2,RYL02,FNG01,Queen doll,9.49,12 inch queen doll with royal garments and crown


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [138]:
sql_req = """SELECT cust_name,
                    (SELECT COUNT(order_num) FROM Orders WHERE Customers.cust_id = Orders.cust_id)
             FROM Customers
"""

DataFrame(curs.execute(sql_req).fetchall(), columns=('cust_name', 'к-сть замовлень'))

,cust_name,к-сть замовлень
0,Village Toys,2
1,Kids Place,0
2,Fun4All,1
3,Fun4All,1
4,The Toy Store,1


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [139]:
sql_req = """SELECT Customers.cust_name,
                    Orders.order_num,
                    Orders.order_date
            FROM Customers
            INNER JOIN Orders ON Orders.cust_id = Customers.cust_id
"""
DataFrame(curs.execute(sql_req).fetchall(), columns=('cust_name', 'order_num', 'order_date'))

,cust_name,order_num,order_date
0,Village Toys,20005,2012-05-01
1,Fun4All,20006,2012-01-12
2,Fun4All,20007,2012-01-30
3,The Toy Store,20008,2012-02-03
4,Village Toys,20009,2012-02-08


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [145]:
sql_req = """SELECT * FROM Products WHERE prod_price < 5
            UNION
            SELECT * FROM Products WHERE prod_price > 5
"""
curs.execute("PRAGMA table_info(Products)")
columns = [row[1] for row in curs.fetchall()]

DataFrame(curs.execute(sql_req).fetchall(), columns=columns)

,prod_id,vend_id,prod_name,prod_price,prod_desc
0,BNBG01,DLL01,Fish bean bag toy,3.49,"Fish bean bag toy, complete with bean bag worm..."
1,BNBG02,DLL01,Bird bean bag toy,3.49,"Bird bean bag toy, eggs are not included"
2,BNBG03,DLL01,Rabbit bean bag toy,3.49,"Rabbit bean bag toy, comes with bean bag carrots"
3,BR01,BRS01,8 inch teddy bear,5.99,"8 inch teddy bear, comes with cap and jacket"
4,BR02,BRS01,12 inch teddy bear,8.99,"12 inch teddy bear, comes with cap and jacket"
5,BR03,BRS01,18 inch teddy bear,11.99,"18 inch teddy bear, comes with cap and jacket"
6,RGAN01,DLL01,Raggedy Ann,4.99,18 inch Raggedy Ann doll
7,RYL01,FNG01,King doll,9.49,12 inch king doll with royal garments and crown
8,RYL02,FNG01,Queen doll,9.49,12 inch queen doll with royal garments and crown


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


1. Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd

In [ ]:
SELECT model, speed, hd
FROM PC
WHERE price < 500

2. Найдите производителей принтеров. Вывести: maker

In [ ]:
SELECT DISTINCT maker
FROM Product
WHERE type = 'Printer'

3. Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
SELECT model, ram, screen
FROM Laptop
WHERE price > 1000

4. Найдите все записи таблицы Printer для цветных принтеров.

In [ ]:
SELECT *
FROM Printer
WHERE color = 'y'

5. Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

In [ ]:
SELECT model, speed, hd
FROM PC
WHERE cd IN ('12x', '24x') AND price < 600

6. Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.

In [ ]:
SELECT DISTINCT Product.maker AS Maker, Laptop.speed
FROM Product
INNER JOIN Laptop ON Laptop.model = Product.model
WHERE Laptop.hd >= 10

7. Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).

In [ ]:
SELECT Product.model, PC.price
FROM Product
INNER JOIN PC ON PC.model = Product.model
WHERE Product.maker = 'B'
UNION
SELECT Product.model, Laptop.price
FROM Product
INNER JOIN Laptop ON Laptop.model = Product.model
WHERE Product.maker = 'B'
UNION
SELECT Product.model, Printer.price
FROM Product
INNER JOIN Printer ON Printer.model = Product.model
WHERE Product.maker = 'B'

8. Найдите производителя, выпускающего ПК, но не ПК-блокноты.

In [ ]:
SELECT maker
FROM Product
WHERE type = 'PC'
EXCEPT 
SELECT maker
FROM Product
WHERE type = 'Laptop'

9. Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker

In [ ]:
SELECT DISTINCT Product.maker
FROM Product
JOIN PC ON PC.model = Product.model
WHERE Product.type = 'PC' AND PC.speed >= 450

10. Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price

In [ ]:
SELECT model, price
FROM Printer
WHERE price = (SELECT MAX(price) FROM Printer)

11. Найдите среднюю скорость ПК.

In [ ]:
SELECT AVG(speed)
FROM PC

12. Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол.

In [ ]:
SELECT AVG(speed)
FROM Laptop
WHERE price > 1000

13. Найдите среднюю скорость ПК, выпущенных производителем A.

In [ ]:
SELECT AVG(PC.speed)
FROM PC
JOIN Product ON Product.model = PC.model
WHERE Product.maker = 'A'

14. Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий.

In [ ]:
SELECT Ships.class, Ships.name, Classes.country
FROM Ships
JOIN Classes ON Classes.class = Ships.class
WHERE numGuns >= 10

15. Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD

In [ ]:
SELECT hd
FROM PC
GROUP BY hd
HAVING COUNT(hd) >= 2

16. Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

In [ ]:
SELECT DISTINCT pc1.model, pc2.model, pc1.speed, pc1.ram
FROM PC AS pc1, PC AS pc2
WHERE pc1.speed = pc2.speed AND pc1.ram = pc2.ram AND pc1.model > pc2.model

17. Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed

In [ ]:
SELECT DISTINCT Product.type, Laptop.model, Laptop.speed
FROM Laptop
JOIN Product ON Product.model = Laptop.model
WHERE speed < ALL(SELECT speed FROM PC)

18. Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price

In [ ]:
SELECT maker, price
FROM Product
JOIN Printer ON Printer.model = Product.model
WHERE color = 'y' AND
Printer.price = (SELECT MIN(price) FROM Printer WHERE color = 'y')

19. Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов.
Вывести: maker, средний размер экрана.

In [ ]:
SELECT maker, AVG(screen)
FROM Product
JOIN Laptop ON Laptop.model = Product.model
GROUP BY maker

20. Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

In [ ]:
SELECT maker, COUNT(model) as Count_Model
FROM Product
WHERE type = 'PC'
GROUP BY maker
HAVING COUNT(model) >= 3

21. Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена.

In [ ]:
SELECT maker, MAX(price)
FROM Product
JOIN PC ON PC.model = Product.model
GROUP BY maker

22. Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена.

In [ ]:
SELECT speed, AVG(price)
FROM PC
GROUP BY speed
HAVING speed > 600

23. Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker

In [ ]:
SELECT Maker
FROM Product
JOIN PC ON PC.model = Product.model
WHERE speed >= 750
INTERSECT
SELECT Maker
FROM Product
JOIN Laptop ON Laptop .model = Product.model
WHERE speed >= 750

24. Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

In [ ]:
SELECT model FROM(
SELECT model, price FROM PC
UNION
SELECT model, price FROM Laptop
UNION
SELECT model, price FROM Printer
) AS table1
WHERE price = (
SELECT MAX(price) FROM(
SELECT model, price FROM PC
UNION
SELECT model, price FROM Laptop
UNION
SELECT model, price FROM Printer) AS table2
)

25. Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

In [ ]:
SELECT DISTINCT maker FROM pc JOIN product on product.model = pc.model WHERE maker
IN(
SELECT DISTINCT maker FROM product WHERE type = 'printer'
) AND speed = (select MAX(speed) FROM pc WHERE ram = (SELECT MIN(ram) FROM pc))
AND ram = (SELECT MIN(ram) FROM pc)

26. Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

In [ ]:
SELECT SUM(prices) / SUM(models) avg_price FROM(
SELECT COUNT(PC.model) AS models, SUM(price) AS prices FROM PC JOIN product ON
product.model = PC.model WHERE maker = 'A'
UNION
SELECT COUNT(Laptop.model), SUM(price) FROM laptop JOIN product ON product.model = laptop.model WHERE maker = 'A'
) this_table

27. Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD

In [ ]:
SELECT maker, AVG(hd) avg_hd
FROM PC
JOIN product ON product.model = PC.model
WHERE maker IN(SELECT DISTINCT maker FROM product WHERE TYPE = 'printer')
GROUP BY maker

28. Используя таблицу Product, определить количество производителей, выпускающих по одной модели.

In [ ]:
SELECT COUNT(maker)
FROM (SELECT maker FROM Product GROUP By maker HAVING COUNT(maker) = 1) AS new_table

29. В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.

In [ ]:
SELECT income_o.point, income_o.[date], inc, out FROM income_o 
LEFT JOIN outcome_o
ON outcome_o.point = income_o.point AND outcome_o.[date] = income_o.[date]
UNION
SELECT outcome_o.point, outcome_o.[date], inc, out FROM income_o
RIGHT JOIN outcome_o ON outcome_o.point = income_o.point AND outcome_o.[date] = income_o.[date]

30. В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).

In [ ]:
SELECT point, [date], SUM(outs), SUM(incs) FROM(
SELECT point, [date], SUM(out) outs, null incs FROM outcome GROUP BY point, [date]
UNION
SELECT point, [date], null, SUM(inc) FROM income GROUP BY point, [date]
) this_table GROUP BY point, [date]